In [17]:
!pip install watson_developer_cloud

import re
import numpy as np
import tensorflow as tf

In [22]:
!ls

checkpoint			     params.ckpt-4.index
cloudclassifier.npy		     params.ckpt-4.meta
clouddata.npz.zip		     testset.npy
cloudtrainingdata.csv		     x_test.npy
params.ckpt-199.data-00000-of-00001  x_train.npy
params.ckpt-199.index		     y_test.npy
params.ckpt-199.meta		     y_train.npy
params.ckpt-4.data-00000-of-00001


In [5]:
x_test = np.load('x_test.npy')
y_test = np.load('y_test.npy')

helper functions to create predictions

In [6]:
def placeholders(n_x, n_y):
    X = tf.placeholder(tf.float32, shape=(n_x, None), name=None)
    Y = tf.placeholder(tf.float32, shape=(n_y, None), name=None)
    
    return X, Y

def forward_prop(X, parameters):
    
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    W3 = parameters['W3']
    b3 = parameters['b3']
    W4 = parameters['W4']
    b4 = parameters['b4']
    
    Z1 = tf.add(tf.matmul(W1,X), b1)                      
    A1 = tf.nn.relu(Z1)                                   
    Z2 = tf.add(tf.matmul(W2,A1), b2)                   
    A2 = tf.nn.relu(Z2)                                   
    Z3 = tf.add(tf.matmul(W3,A2), b3)  
    A3 = tf.nn.relu(Z3)
    Z4 = tf.add(tf.matmul(W4,A3), b4)
    
    return Z4

def compute_accuracy(pred, testdata):
    count = 0
    for i in range(len(pred)):
        if pred[i]==testdata[i][1]:
            count += 1
    return float(count)/len(pred)

In [12]:
from watson_developer_cloud import NaturalLanguageClassifierV1

natural_language_classifier = NaturalLanguageClassifierV1(
    username='YOUR NLC USERNAME',
    password='YOUR NLC PASSWORD')

### Find the classifier made in the notebook naturallanguageclassifier.ipynb
copy its id to the variable CLASSIFIER_ID

In [ ]:
natural_language_classifier.list_classifiers().get_result()

In [ ]:
CLASSIFIER_ID = 'ENTER CLASSIFIER ID'

check classifier availability. Classifier should be available and ready to take classifier requests

In [15]:
natural_language_classifier.get_classifier(CLASSIFIER_ID).get_result()

{'classifier_id': '9b88dax497-nlc-199',
 'created': '2019-01-31T19:03:58.095Z',
 'language': 'en',
 'name': 'think Cloud Classifier',
 'status': 'Available',
 'status_description': 'The classifier instance is now available and is ready to take classifier requests.',
 'url': 'https://gateway.watsonplatform.net/natural-language-classifier/api/v1/classifiers/9b88dax497-nlc-199'}

In [18]:
testset = np.load('testset.npy')
watsonpred = []
for i in testset:
    x = natural_language_classifier.classify(CLASSIFIER_ID,re.sub(' +',' '," ".join(re.split(r'[^\w]', re.sub(re.compile("/\*.*?\*/",re.DOTALL ) ,"" ,i[0]))))[0:2048])
    watsonpred.append(x.get_result()['top_class'])

In [9]:
with tf.Session() as sess:    
    X, Y = placeholders(x_test.shape[0], y_test.shape[0])
    saver = tf.train.import_meta_graph('params.ckpt-199.meta')
    saver.restore(sess,tf.train.latest_checkpoint('./'))
    parameters = {"W1": sess.run('W1:0'),
                  "b1": sess.run('b1:0'),
                  "W2": sess.run('W2:0'),
                  "b2": sess.run('b2:0'),
                  "W3": sess.run('W3:0'),
                  "b3": sess.run('b3:0'),
                  "W4": sess.run('W4:0'),
                  "b4": sess.run('b4:0')}
    Z4 = forward_prop(X, parameters)
    
    correct_prediction = tf.equal(tf.argmax(Z4), tf.argmax(Y))
    
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Test Accuracy Watson NLC API: " + str(compute_accuracy(watsonpred,testset)))
    print ("Test Accuracy TensorFlow:", accuracy.eval({X: x_test, Y: y_test}))

INFO:tensorflow:Restoring parameters from ./params.ckpt-199
Test Accuracy: 0.903448
